## Import the required libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium

## Create the GeoDataFrame of Hermosillo
First, We read the file where the information was located. Then, We filtered the data to get the information of Hermosillo. We also transformed the polygon type to manage decimals.

In [ ]:
son_df = gpd.read_file('data/26sonora/26-SONORA/COLONIA.dbf')
hillo_df = son_df[son_df.MUNICIPIO == 49]
hillo_df.geometry = hillo_df.geometry.to_crs(epsg=4326) # to manage decimals
hillo_df.head(3)


## Get the locations from the web page in order to convert it.
In this case, We scrapped the data from the web page of the bussiness. Then, We identified the places of interest.
We also used the shapely library to convert the information into a Point Geometry Type. For that goal, We worked with the lat and long.

In [ ]:
coffee_loc = pd.read_csv('data/hillo_coffee_loc.csv').drop(columns=['id'])
coffee_loc['geometry'] = coffee_loc.apply(lambda row: Point((row.lng, row.lat)), axis=1)
coffee_loc.head()

## Create the GeoDataFrame of the coffee shops
We used GeoDataFrame to transform the data. We define the epsg as 4326 to work with decimals.

In [ ]:
hillo_coffee_crs = 'epsg:4326'
hillo_coffee_geo = gpd.GeoDataFrame(coffee_loc,
crs = hillo_coffee_crs,
geometry = coffee_loc.geometry)
hillo_coffee_geo.head()

## Joined the data of Hermosillo and the coffee shops locations
We intersected the data to get the suburbs where there are coffe shops in Hermosillo.

In [ ]:
hillo_coffee_gdf = gpd.sjoin(hillo_df, hillo_coffee_geo, predicate='intersects')
hillo_coffee_gdf.head(2)

## Get the total coffee shops by suburb

In [ ]:
coffee_counts = hillo_coffee_gdf.groupby('NOMBRE').index_right.count().reset_index(level=0)
coffee_counts.columns = ['NOMBRE','TOTAL']
coffee_counts.head()

In [ ]:
hillo_coffee_counts = pd.merge(hillo_coffee_gdf, coffee_counts, on='NOMBRE')
hillo_coffee_counts.head(2)

## Show a Map of Hermosillo, Sonora
In order to do this, We used folium. We center the map with the information provided for the business.
The coords to center the data were, lat: 29.098, lng: -110.950.

In [ ]:
hillo_map = folium.Map(location=[29.098, -110.950], zoom_start=12)

display(hillo_map)

## Add characteristics to create an interactive Choropleth Map
Choropleth can be easily created by binding the data between Pandas DataFrames/Series and Geo/TopoJSON geometries.

In [ ]:
hillo_map.choropleth(
    geo_data=hillo_coffee_counts,
    name='geometry',
    data=hillo_coffee_counts,
    columns=['NOMBRE','TOTAL'],
    key_on='feature.properties.NOMBRE',
    fill_color='OrRd',
    fill_opacity=0.75,
    line_opacity=0.5,
    legend_name='Number of Coffee Shops by Suburb'
)

## Show the Interactive Choropleth Map

In [ ]:
folium.LayerControl().add_to(hillo_map)
display(hillo_map)

## Save map as an html file

In [ ]:
hillo_map.save('coffee_hillo.html')